In [1]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from PIL import Image
from datasets import load_dataset
from anls_star import anls_score
from fastprogress.fastprogress import progress_bar
import requests
import torch
import numpy as np

In [2]:
ds = load_dataset("danjacobellis/docvqa",split='validation')
model_id = "google/paligemma-3b-ft-docvqa-896"
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
    revision="bfloat16",
)
processor = AutoProcessor.from_pretrained(model_id)

Resolving data files:   0%|          | 0/57 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/57 [00:00<?, ?it/s]

/home/dgj335/.local/lib/python3.10/site-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def compute_score(item,res):
    prompt = item['question']
    image = item['image'].resize((res, res), resample=Image.Resampling.LANCZOS).convert("RGB")
    image = image.resize((896, 896), resample=Image.Resampling.LANCZOS)
    model_inputs = processor(text=prompt, images=image, return_tensors="pt").to(model.device)
    input_len = model_inputs["input_ids"].shape[-1]
    with torch.inference_mode():
        generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
        generation = generation[0][input_len:]
        pred = processor.decode(generation, skip_special_tokens=True)
    score, _ = anls_score(item['answer'], pred, return_gt=True)
    return score

In [4]:
scores = []
for res in [224,448,896]:
    scores.append([])
    for item in progress_bar(ds):
        score = compute_score(item,res)
        scores[-1].append(score)
[np.mean(s) for s in scores]

/home/dgj335/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Attempt to open cnn_infer failed: handle=0 error: libcudnn_cnn_infer.so.8: cannot open shared object file: No such file or directory (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:78.)
  return F.conv2d(input, weight, bias, self.stride,
/home/dgj335/.local/lib/python3.10/site-packages/anls_star/anls_star.py:513: UserWarning: Treating ground truth as a list of options. This is a compatibility mode for ST-VQA-like datasets.
  warnings.warn(


[0.4360673981279203, 0.9627833875779456, 0.9985880439163821]